In [59]:
import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="NanumGothic")
matplotlib.rcParams['axes.unicode_minus'] = False

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 23s (452 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123623 files and direc

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [2]:
DATA_PATH = "/content/drive/MyDrive/final/data/"
DATA_PATH

'/content/drive/MyDrive/final/data/'

- 시드값

In [3]:
SEED = 42

- 데이터 불러오기

In [78]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [79]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2916), (12225, 2916))

- 정답 데이터

In [80]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# 결측치 처리

In [81]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0


In [82]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0


# 특성 공학(Feature Engineering)

- ID 변수 제외

In [83]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2915), (12225, 2915))

- 추가 피처 만들어 보기

In [84]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_지점코드_") ]

train_ft["지점코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["지점코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["지점코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["지점코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["지점코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["지점코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2918), (12225, 2918))

In [85]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_중분류_") ]

train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2921), (12225, 2921))

In [86]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_18시이전구매_") ]

train_ft["18시이전구매_구매금액_std"] = train_ft[cols].std(axis=1)
train_ft["18시이전구매_구매금액_skew"] = train_ft[cols].skew(axis=1)
train_ft["18시이전구매_구매금액_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["18시이전구매_구매금액_std"] = test_ft[cols].std(axis=1)
test_ft["18시이전구매_구매금액_skew"] = test_ft[cols].skew(axis=1)
test_ft["18시이전구매_구매금액_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2924), (12225, 2924))

In [87]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_18시이후구매_") ]

train_ft["18시이후구매_구매금액_std"] = train_ft[cols].std(axis=1)
train_ft["18시이후구매_구매금액_skew"] = train_ft[cols].skew(axis=1)
train_ft["18시이후구매_구매금액_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["18시이후구매_구매금액_std"] = test_ft[cols].std(axis=1)
test_ft["18시이후구매_구매금액_skew"] = test_ft[cols].skew(axis=1)
test_ft["18시이후구매_구매금액_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2927), (12225, 2927))

In [88]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_대분류_") ]

train_ft["대분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["대분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["대분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["대분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["대분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["대분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2930), (12225, 2930))

In [89]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_브랜드코드_") ]

train_ft["브랜드코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["브랜드코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["브랜드코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["브랜드코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["브랜드코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["브랜드코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2933), (12225, 2933))

In [90]:
cols = [ col for col in train_ft.columns if col.endswith("요일_구매비율") ]

train_ft["요일별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["요일별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["요일별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["요일별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["요일별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["요일별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2936), (12225, 2936))

In [91]:
cols = [ col for col in train_ft.columns if col.endswith("월_구매비율") ]

train_ft["월별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["월별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["월별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["월별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["월별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["월별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2939), (12225, 2939))

In [92]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2942), (12225, 2942))

In [93]:
cols = [ col for col in train_ft.columns if col.endswith("분기_구매비율") ]

train_ft["분기별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["분기별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["분기별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["분기별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["분기별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["분기별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2945), (12225, 2945))

In [94]:
cols = [ col for col in train_ft.columns if col.endswith("반기_구매비율") ]

train_ft["반기별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["반기별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2946), (12225, 2946))

In [95]:
cols = [ col for col in train_ft.columns if col.endswith("년_구매비율") ]

train_ft["년도별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["년도별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2947), (12225, 2947))

## Feature Encoding

In [96]:
train_ft["첫구매날짜"] = pd.to_datetime(train_ft["첫구매날짜"])
train_ft["마지막구매날짜"] = pd.to_datetime(train_ft["마지막구매날짜"])
test_ft["첫구매날짜"] = pd.to_datetime(test_ft["첫구매날짜"])
test_ft["마지막구매날짜"] = pd.to_datetime(test_ft["마지막구매날짜"])

train_ft["첫구매년도"] = train_ft["첫구매날짜"].dt.year
train_ft["첫구매월"] = train_ft["첫구매날짜"].dt.month
train_ft["마지막구매년도"] = train_ft["마지막구매날짜"].dt.year
train_ft["마지막구매월"] = train_ft["마지막구매날짜"].dt.month

test_ft["첫구매년도"] = test_ft["첫구매날짜"].dt.year
test_ft["첫구매월"] = test_ft["첫구매날짜"].dt.month
test_ft["마지막구매년도"] = test_ft["마지막구매날짜"].dt.year
test_ft["마지막구매월"] = test_ft["마지막구매날짜"].dt.month

train_ft.shape, test_ft.shape

((14940, 2951), (12225, 2951))

- 범주형 피처
    - 주구매지점, 선호브랜드코드, 주구매시간대 : onehot
    - 주구매_중분류, 주구매_대분류 : onehot -> PCA

In [35]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_중분류,232
주구매_대분류,28


In [37]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.4 MB/s eta 0:00:00


In [97]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점", "선호브랜드코드", "주구매_중분류", "주구매_대분류", "주구매시간대"]

oh_enc.fit(train_ft[oh_enc_cols])

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

<ipython-input-97-7001e00c8c5c>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
<ipython-input-97-7001e00c8c5c>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
<ipython-input-97-7001e00c8c5c>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

((14940, 3874), (12225, 3874))

- 문자열 피처 삭제

In [98]:
encoded_columns = oh_enc.get_feature_names_out(oh_enc_cols)
encoded_columns

array(['주구매지점_A112000', '주구매지점_A144000', '주구매지점_A202000', '주구매지점_A373000',
       '선호브랜드코드_5100', '선호브랜드코드_5101', '선호브랜드코드_5102', '선호브랜드코드_5103',
       '선호브랜드코드_5104', '선호브랜드코드_5105', '선호브랜드코드_5106', '선호브랜드코드_5107',
       '선호브랜드코드_5108', '선호브랜드코드_5109', '선호브랜드코드_5110', '선호브랜드코드_5111',
       '선호브랜드코드_5112', '선호브랜드코드_5113', '선호브랜드코드_5114', '선호브랜드코드_5115',
       '선호브랜드코드_5116', '선호브랜드코드_5117', '선호브랜드코드_5118', '선호브랜드코드_5119',
       '선호브랜드코드_5120', '선호브랜드코드_5121', '선호브랜드코드_5122', '선호브랜드코드_5124',
       '선호브랜드코드_5125', '선호브랜드코드_5126', '선호브랜드코드_5127', '선호브랜드코드_5128',
       '선호브랜드코드_5129', '선호브랜드코드_5130', '선호브랜드코드_5131', '선호브랜드코드_5132',
       '선호브랜드코드_5133', '선호브랜드코드_5135', '선호브랜드코드_5136', '선호브랜드코드_5137',
       '선호브랜드코드_5138', '선호브랜드코드_5140', '선호브랜드코드_5141', '선호브랜드코드_5142',
       '선호브랜드코드_5143', '선호브랜드코드_5144', '선호브랜드코드_5145', '선호브랜드코드_5146',
       '선호브랜드코드_5147', '선호브랜드코드_5148', '선호브랜드코드_5149', '선호브랜드코드_5150',
       '선호브랜드코드_5152', '선호브랜드코드_5153', '선호브랜드코드_5154', '선호브랜드코드_5155',
  

In [99]:
train_ft = train_ft.drop(columns=oh_enc_cols)
test_ft = test_ft.drop(columns=oh_enc_cols)

train_ft.shape, test_ft.shape

((14940, 3869), (12225, 3869))

In [100]:
cols = train_ft.select_dtypes(["object", "datetime"]).columns.tolist()
cols

['첫구매날짜', '마지막구매날짜']

In [101]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)

train_ft.shape, test_ft.shape

((14940, 3867), (12225, 3867))

In [102]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [103]:
train_ft.select_dtypes("datetime").columns, test_ft.select_dtypes("datetime").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [104]:
train_ft.isin([np.inf, -np.inf]).any().sum(), test_ft.isin([np.inf, -np.inf]).any().sum()

(0, 0)

In [105]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()

(0, 0)

In [106]:
train_ft.shape, test_ft.shape

((14940, 3867), (12225, 3867))

In [107]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,총방문일수,백화점이용기간,구매주기,주말방문일수,일별평균구매건수,구매간격_표준편차,평균구매요일,주구매요일,월요일_구매비율,화요일_구매비율,...,주구매시간대_11,주구매시간대_12,주구매시간대_13,주구매시간대_14,주구매시간대_15,주구매시간대_16,주구매시간대_17,주구매시간대_18,주구매시간대_19,주구매시간대_20
0,0.035398,0.699725,0.153409,0.037037,0.079365,0.135386,0.500000,0.333333,0.071429,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.088496,0.966942,0.085227,0.018519,0.142857,0.069876,0.384921,0.500000,0.047619,0.261905,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.238938,0.977961,0.028409,0.203704,0.122078,0.029881,0.388889,0.000000,0.254902,0.186275,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.393805,0.947658,0.011364,0.351852,0.150794,0.018673,0.513514,0.666667,0.075676,0.113514,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.101770,0.859504,0.068182,0.111111,0.071429,0.071207,0.643519,0.666667,0.083333,0.055556,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# 차원축소

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

model = XGBClassifier(random_state=SEED)
model.fit(train_ft, target)

scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.7095029339664529

## PCA (주구매_중분류, 주구매_대분류)

In [69]:
pca_columns = [col for col in train_ft if any(prefix in col for prefix in ['주구매_중분류', '주구매_대분류'])]
len(pca_columns)

272

In [71]:
from sklearn.decomposition import PCA

pca = PCA(50, random_state=SEED)
pca_result = pca.fit_transform(train_ft[pca_columns])
pca_columns_result = ['PCA_' + str(i+1) for i in range(50)]
train_ft[pca_columns_result] = pca_result

scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores), train_ft.shape

<ipython-input-71-4f283e1de2b5>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ft[pca_columns_result] = pca_result
<ipython-input-71-4f283e1de2b5>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ft[pca_columns_result] = pca_result
<ipython-input-71-4f283e1de2b5>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `n

(0.7080874678793736, (14940, 3917))

In [76]:
pca_columns = [col for col in test_ft if any(prefix in col for prefix in ['주구매_중분류', '주구매_대분류'])]
len(pca_columns)

272

In [77]:
pca = PCA(50, random_state=SEED)
pca_result = pca.transform(test_ft[pca_columns])
pca_columns_result = ['PCA_' + str(i+1) for i in range(50)]
test_ft[pca_columns_result] = pca_result

scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores), test_ft.shape

AttributeError: 'PCA' object has no attribute 'components_'

# cv 점수 확인해보기

In [108]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [109]:
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0.7095029339664529

# 모델 학습

In [110]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=SEED)
model.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

# 테스트 데이터 예측

In [111]:
pred = model.predict(test_ft)
pred

array([0, 0, 0, ..., 0, 0, 0])

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [112]:
submit["target"] = pred
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


- 예측 결과를 csv 파일로 저장하여 제출

In [113]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)